In [1]:
# do basic imports and unpack McMurdo data

from pmagpy import ipmag
reload(ipmag)
from pmagpy import pmag
from programs import new_builder as nb
from programs import data_model3
reload(data_model3)
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from programs.new_builder import Contribution

import pmagpy.controlled_vocabularies3 as cv



/Users/nebula/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


got full_df
got parsed_df
got full_df
got parsed_df


In [2]:
dir_path = os.path.join(os.getcwd(), '3_0', 'Megiddo')
con = Contribution(dir_path)

loc_dm = con.tables['locations'].data_model.dm['locations']
loc_df = con.tables['locations'].df
site_dm = con.tables['sites'].data_model.dm['sites']
site_df = con.tables['sites'].df
samp_df = con.tables['samples'].df
samp_dm = con.tables['samples'].data_model.dm['samples']
spec_df = con.tables['specimens'].df
spec_dm = con.tables['specimens'].data_model.dm['specimens']
age_df = con.tables['ages'].df
age_dm = con.tables['ages'].data_model.dm['ages']


current_con = con
current_df = age_df
current_dm = age_dm
# replace NaN with None
current_dm = current_dm.where((pd.notnull(current_dm)), None)
cols = current_df.columns


#vocab.get_controlled_vocabularies()

# mess up some validations for locations
#current_df.loc['Tel Hazor']['lat_s'] = 400.
#current_df['dir_inc'] = 5
#current_con.tables.pop('sites')

# mess up some validations for sites
#current_df.pop('age')
#current_df['dir_tilt_correction'] = 1
#current_df['dir_tilt_correction'] = 'one'

# mess up some validations for samples
#current_df.pop('citations')
#current_df.iloc[0].lon = 600.
#current_df.iloc[1].lat = 'hello'

current_df.head()

-W- No such file: /Users/nebula/Python/PmagPy/3_0/Megiddo/images.txt


,age,age_high,age_low,age_unit,citations,description,location,method_codes,site
age,,,,,,,,,
-740,-740,-732,-800,Years Cal AD (+/-),This study,"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz05
-740,-740,-732,-800,Years Cal AD (+/-),This study,"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz06
-850,-850,-800,-900,Years Cal AD (+/-),This study,"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz07
-950,-950,-900,-1000,Years Cal AD (+/-),This study,"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz09
-950,-950,-900,-1000,Years Cal AD (+/-),This study,"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz10


In [3]:
import pmagpy.controlled_vocabularies3 as cv
reload(cv)
vocab = cv.Vocabulary()
vocabulary, possible_vocabulary = vocab.get_controlled_vocabularies()



-I- Importing controlled vocabularies from https://earthref.org


In [12]:
# check column validity
invalid_cols = [col for col in cols if col not in current_dm.index]


def requiredUnless(col_name, df, arg, *args):
    arg_list = arg.split(",")
    arg_list = [arg.strip('"') for arg in arg_list]
    msg = ""
    for a in arg_list:
        if "." in a:
            continue
        if a not in df.columns:
            msg += "{} is required unless {} is present.  ".format(col_name, a)
    if msg:
        return msg
    else:
        return None
    return None


def requiredUnlessTable(col_name, df, arg, *args):
    """
    Col_name must be present in df unless
    arg (table_name) is present in contribution
    """
    table_name = arg
    if col_name in df.columns:
        return None
    elif table_name in current_con.tables:
        return None
    else:
        #print "{} is required unless table {} is present".format(col_name, table_name)
        return "{} is required unless table {} is present".format(col_name, table_name)

    
def requiredIfGroup(col_name, df, arg, *args):
    """
    Col_name is required if other columns of 
    the group arg are present.
    """
    group_name = arg
    groups = set()
    columns = df.columns
    for col in columns:
        if col not in current_dm.index:
            continue
        group = current_dm.loc[col]['group']
        groups.add(group)
    if group_name in groups:
        if col_name in columns:
            return None
        else:
            #print "{} is required if column group {} is used".format(col_name, group_name)
            return "{} is required if column group {} is used".format(col_name, group_name)
    return None


def required(col_name, df, arg):
    #print df
    if col_name in df.columns:
        return None
    else:
        return "{} is required".format(col_name)
    

def check_max(row, col_name, arg, *args):
    #print 'check_max'
    cell_value = row[col_name]
    if not cell_value:
        return None
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
    arg = float(arg)
    try:
        if float(cell_value) <= float(arg):
            return None
        else:
            return "{} must be <= {}".format(str(cell_value), str(arg))
    # this happens when the value isn't a float (an error which will be caught elsewhere)
    except ValueError:
        return None

def check_min(row, col_name, arg, *args):
    print 'check_min'
    cell_value = row[col_name]
    print 'cell_value', cell_value
    if not cell_value:
        return None
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
    print 'arg', arg 
    try:
        if float(cell_value) >= float(arg):
            return None
        else:
            return "{} must be >= {}".format(str(cell_value), str(arg))
    # this happens when the value isn't a float (an error which will be caught elsewhere)
    except ValueError:
        return None

def cv(row, col_name, arg, current_data_model):
    cell_value = row[col_name]
    if not cell_value:
        return None
    if cell_value.lower() in [v.lower() for v in vocabulary[col_name]]:
        return None
    else:
        return "{} is not in {}".format(cell_value, arg)
        

# validate presence
presence_operations = {"required": required, "requiredUnless": requiredUnless,
                       "requiredIfGroup": requiredIfGroup, 
                       'requiredUnlessTable': requiredUnlessTable}
# validate values
value_operations = {"max": check_max, "min": check_min, "cv": cv}

def split_func(string):
    """
    Take a string like 'requiredIf("arg_name")'
    return the function name and the argument:
    (requiredIf, arg_name)
    """
    ind = string.index("(")
    return string[:ind], string[ind+1:-1].strip('"')


def test_type(value, value_type):
    if not value:
        return None
    if value_type == "String":
        if str(value) == value:
            return None
        else:
            return "should be string"
    elif value_type == "Number":
        try:
            float(value)
            return None
        except ValueError:
            return "should be a number"
    elif value_type == "Integer":
        if isinstance(value, str):
            if str(int(value)) == value:
                return None
            else:
                return "should be an integer"
        else:
            if int(value) == value:
                return None
            else:
                return "should be an integer"
    else:
        return None
    #String
    # Number
    #Integer
    #List
    #Matrix
    #Dictionary
    

#current_df["presence_pass_age_requiredUnless"] = ''
#print current_df["presence_pass_age_requiredUnless"]
for validation_name, validation in current_dm.iterrows():
    #if validation_name == "age":
    #    continue
    
    # test types first
    #array([u'Number', u'String', u'List', u'Matrix', u'Integer', u'Dictionary'], dtype=object)
    value_type = validation['type']
    #print 'value_type', value_type
    if validation_name in current_df.columns:
        output = current_df[validation_name].apply(test_type, args=(value_type,))
        current_df["type_" + validation_name] = output
    
    
    val_list = validation['validations']
    if not val_list:
        continue
    for num, val in enumerate(val_list):
        #if num > 0:
        #    continue
        # split val into function & arg
        func_name, arg = split_func(val)
        # first validate for presence
        if func_name in presence_operations:
            func = presence_operations[func_name]
            grade = func(validation_name, current_df, arg)
            pass_col_name = "presence_pass_" + validation_name + "_" + func.__name__
            current_df[pass_col_name] = grade
            #print current_df[pass_col_name]

        # then validate for correct values
        elif func_name in value_operations:
            func = value_operations[func_name]
            if validation_name in current_df.columns:
                #print val, "!!"
                #grade = current_df[validation_name].apply(func, args=(validation_name, current_df, arg))
                grade = current_df.apply(func, args=(validation_name, arg, current_dm), axis=1)
                current_df["pass_" + validation_name + "_" + str(num)] = grade.astype(object)
                

# need to test value types....                
                
                
#print current_df["presence_pass_age_requiredUnless"]

#current_df["presence_pass_" + validation_name] = 'hi'
#print current_df["presence_pass_" + validation_name]

  # check that values pass validation
# required()
# requiredUnless
# min
# max
# cv (controlled vocab)
# in
# requiredIfGroup
# type
# sv (suggested vocab)



# re-do upload_magic to use contribution-level (??)

# first, do validations on each table in the contribution
# this will include removing unneded data (RmKeys from old upload_magic)
# this will also include checking everything against the data model (strings are strings, etc.)g

# next, do cross-contribution validations (all specimens exist, etc.)

# next, splat out each table into a file and wrap it up.  give it a sensible name.  

pass_cols = current_df.columns.str.match("^pass_")
present_cols = current_df.columns.str.match("^presen")
type_cols = current_df.columns.str.match("^type_")
val_cols = np.where([bool(v) for v in pass_cols], [bool(v) for v in pass_cols], [bool(v) for v in present_cols])

present_col_names = current_df.columns[present_cols]
current_df[present_col_names].dropna(how='all', axis=1).head()

current_df[current_df.columns[type_cols]].dropna(how='all', axis=1).head()



check_min
cell_value -732
arg -800
check_min
cell_value -732
arg -800
check_min
cell_value -800
arg -900
check_min
cell_value -900
arg -1000
check_min
cell_value -900
arg -1000
check_min
cell_value -1000
arg -1100
check_min
cell_value -1250
arg -1350
check_min
cell_value -1450
arg -1650
check_min
cell_value -2000
arg -2300
check_min
cell_value -2500
arg -2800
check_min
cell_value -1400
arg -1500
check_min
cell_value -1700
arg -1900
check_min
cell_value -732
arg -800
check_min
cell_value -800
arg -900
check_min
cell_value -850
arg -950
check_min
cell_value -950
arg -1050
check_min
cell_value -1100
arg -1150
check_min
cell_value -1100
arg -1150
check_min
cell_value -2900
arg -3100
check_min
cell_value -2800
arg -2600
check_min
cell_value -1180
arg -1130
check_min
cell_value -1200
arg -1300
check_min
cell_value -1300
arg -1400
check_min
cell_value -1300
arg -1400
check_min
cell_value -800
arg -900
check_min
cell_value -800
arg -900
check_min
cell_value -850
arg -950
check_min
cell_value -

""
age
-740
-740
-850
-950
-950


In [5]:
current_df[current_df.columns[present_cols]].dropna(how='all', axis=1).head()

""
age
-740
-740
-850
-950
-950


In [8]:
dropped_df = current_df[current_df.columns[pass_cols]].dropna(how='all', axis=1)#.head()
dropped_df.dropna(how='all', axis=0).head()


,pass_age_high_0,pass_age_low_0
age,,
-2700,-2800 must be >= -2600,-2600 must be <= -2800.0
-1130,-1180 must be >= -1130,-1130 must be <= -1180.0


In [14]:
current_df.loc['-2700']

,age,age_high,age_low,age_unit,citations,description,location,method_codes,site,type_age,...,type_age_unit,pass_age_unit_0,presence_pass_age_unit_requiredIfGroup,type_citations,presence_pass_citations_required,type_description,type_location,type_method_codes,presence_pass_method_codes_required,type_site
age,,,,,,,,,,,,,,,,,,,,,
-2700,-2700,-2500,-2800,Years Cal AD (+/-),This study,"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz20,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2700,-2700,-2800,-2600,Years Cal AD (+/-),This study,Tel-Magiddo chronology. Megido VI published re...,Tel Megiddo,GM-C14:GM-CC-ARCH,mgj06,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
current_dm.loc['age_high']

description                                           Age, High range
examples                                                         None
group                                                             Age
label                                                        Age High
notes                                                            None
position                                                           14
previous_columns    [{u'column': u'age_range_high', u'table': u'er...
type                                                           Number
unit                                                           Custom
validations                                          [min("age_low")]
Name: age_high, dtype: object